In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('dataOneHot/', one_hot=True)

W0813 23:04:37.070080 140119469496064 deprecation.py:323] From <ipython-input-1-2bf31ac605dd>:6: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0813 23:04:37.071257 140119469496064 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0813 23:04:37.072701 140119469496064 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be remove

Extracting dataOneHot/train-images-idx3-ubyte.gz


W0813 23:04:37.277030 140119469496064 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0813 23:04:37.319545 140119469496064 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting dataOneHot/train-labels-idx1-ubyte.gz
Extracting dataOneHot/t10k-images-idx3-ubyte.gz
Extracting dataOneHot/t10k-labels-idx1-ubyte.gz


In [2]:
learning_rate_RMSProp = 0.02
learning_rate_GradientDescent = 0.5
num_epochs = 100
batch_size = 256
display_step = 5
input_size = 784 # 28x28=784
hidden1_size = 128
hidden2_size = 64

In [3]:
x = tf.placeholder(tf.float32, shape=[None, input_size])
y = tf.placeholder(tf.float32, shape=[None, 10])

In [4]:
# autoencoder 구조 정의
def build_autoencoder(x):
    # encoding: 784->128->64
    Wh_1 = tf.Variable(tf.random_normal([input_size, hidden1_size]))
    bh_1 = tf.Variable(tf.random_normal([hidden1_size]))
    H1_output = tf.nn.sigmoid(tf.matmul(x,Wh_1) + bh_1)
    Wh_2 = tf.Variable(tf.random_normal([hidden1_size, hidden2_size]))
    bh_2 = tf.Variable(tf.random_normal([hidden2_size]))
    H2_output = tf.nn.sigmoid(tf.matmul(H1_output,Wh_2) + bh_2)
    # decoding: 64->128->784
    Wh_3 = tf.Variable(tf.random_normal([hidden2_size, hidden1_size]))
    bh_3 = tf.Variable(tf.random_normal([hidden1_size]))
    H3_output = tf.nn.sigmoid(tf.matmul(H2_output,Wh_3) + bh_3)
    Wo = tf.Variable(tf.random_normal([hidden1_size, input_size]))
    bo = tf.Variable(tf.random_normal([input_size]))
    X_reconstructed = tf.nn.sigmoid(tf.matmul(H3_output,Wo) + bo)
    
    return X_reconstructed, H2_output

In [5]:
# softmax 분류기 정의
def build_softmax_classification(x):
    W_softmax = tf.Variable(tf.zeros([hidden2_size, 10])) # input_size 대신 hidden2_size. 인코딩된 값을 입력받을 것이므로...
    b_softmax = tf.Variable(tf.zeros([10]))
    y_pred = tf.nn.softmax(tf.matmul(x, W_softmax) + b_softmax)
    
    return y_pred

In [6]:
y_pred, extracted_features = build_autoencoder(x)
y_true = x
y_pred_softmax = build_softmax_classification(extracted_features)

In [7]:
# 1: Pre-Training: MNIST 데이터 재구축을 목적으로 하는 손실 함수와 옵티마이저를 정의합니다.
pretraining_loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
pretraining_train_step = tf.train.RMSPropOptimizer(learning_rate_RMSProp).minimize(pretraining_loss)

# 2: Fine-Tuning: MNIST 데이터 분류를 목적으로하는 손실 함수와 옵티마이저를 정의합니다.
finetuning_loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(y_pred_softmax), reduction_indices=[1])) # cross-entropy loss 함수
finetuning_train_step = tf.train.GradientDescentOptimizer(learning_rate_GradientDescent).minimize(finetuning_loss)

W0813 23:04:37.723393 140119469496064 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0813 23:04:37.800377 140119469496064 deprecation.py:506] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/python/training/rmsprop.py:119: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    total_batch = int(mnist.train.num_examples/batch_size)
    
    # step1: MNIST 데이터 재구축을 위한 오토인코더 최적화 (pre-training)
    for epoch in range(num_epochs):
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, pretraining_loss_print = sess.run([pretraining_train_step, pretraining_loss], feed_dict={x: batch_xs})
        if epoch%display_step == 0:
            print('Epoch: %d, Pre-training Loss: %f' %((epoch+1), pretraining_loss_print))
    print('Step 1: MNIST 데이터 재구축을 위한 오토인코더 최적화 완료')
    
    # step2: MNIST 데이터 분류를 위한 오토인코더+Softmax 분류기 최적화 (fine-tuning)
    for epoch in range(num_epochs+100):
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, finetuning_loss_print = sess.run([finetuning_train_step, finetuning_loss], feed_dict={x: batch_xs, y:batch_ys})
        if epoch%display_step == 0:
            print('Epoch: %d, Fine-tuning Loss: %f' %((epoch+1), finetuning_loss_print))
    print('Step 2: MNIST 데이터 분류를 위한 오토인코더+softmax 분류기 최적화 완료')
    
    # step3: 정확도 출력
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_pred_softmax,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print('Accuracy: %f' %sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))

Epoch: 1, Pre-training Loss: 0.191352
Epoch: 6, Pre-training Loss: 0.072684
Epoch: 11, Pre-training Loss: 0.059698
Epoch: 16, Pre-training Loss: 0.054612
Epoch: 21, Pre-training Loss: 0.047236
Epoch: 26, Pre-training Loss: 0.043226
Epoch: 31, Pre-training Loss: 0.038424
Epoch: 36, Pre-training Loss: 0.036469
Epoch: 41, Pre-training Loss: 0.034315
Epoch: 46, Pre-training Loss: 0.030423
Epoch: 51, Pre-training Loss: 0.030959
Epoch: 56, Pre-training Loss: 0.027994
Epoch: 61, Pre-training Loss: 0.026473
Epoch: 66, Pre-training Loss: 0.027474
Epoch: 71, Pre-training Loss: 0.024613
Epoch: 76, Pre-training Loss: 0.022621
Epoch: 81, Pre-training Loss: 0.020996
Epoch: 86, Pre-training Loss: 0.020147
Epoch: 91, Pre-training Loss: 0.019124
Epoch: 96, Pre-training Loss: 0.018463
Step 1: MNIST 데이터 재구축을 위한 오토인코더 최적화 완료
Epoch: 1, Fine-tuning Loss: 0.446166
Epoch: 6, Fine-tuning Loss: 0.239878
Epoch: 11, Fine-tuning Loss: 0.202973
Epoch: 16, Fine-tuning Loss: 0.168865
Epoch: 21, Fine-tuning Loss: 0.12